<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/SalesMail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mecab-python3
!pip install mojimoji

     |████████████████████████████████| 15.9MB 30.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/67/f0/6e/03e5c5017afc8230e19abeacad57189138207fea2f5dd71880
Successfully built mojimoji


In [0]:
import re
import MeCab
import mojimoji

def serialize(words):
    #大文字統一
    words = words.upper()

    #英数字のみ半角統一
    words = mojimoji.zen_to_han(words, kana=False)
      
    #電話番号削除
    #(稼働時間と形式が重複する為)
    if len(re.findall('[0-9]{3}-[0-9]{4}-[0-9]{4}',words)) > 0:
      tel_list = re.findall('[0-9]{3}-[0-9]{4}-[0-9]{4}',words)
      if len(tel_list) > 0:
        for tel in tel_list:
          words = words.replace(tel,'')

    #チルダ統一1
    ser_word = ["〜","～", "-","－","から","MAX"]
    for word in ser_word:
      words = words.replace(word,"~")
      
    #稼働時間削除
    if len(re.findall('[0-9]{3}H~[0-9]{3}H',words)) > 0:
      kado = re.findall('[0-9]{3}H~[0-9]{3}H',words)[0]
      words = words.replace(kado,"")
    if len(re.findall('[0-9]{3}~[0-9]{3}H',words)) > 0:
      kado = re.findall('[0-9]{3}~[0-9]{3}H',words)[0]
      words = words.replace(kado,"")
    if len(re.findall('[0-9]{3}~[0-9]{3}',words)) > 0:
      kado = re.findall('[0-9]{3}~[0-9]{3}',words)[0]
      words = words.replace(kado,"")
      
    #スキル見合い
    ser_word = ["スキル見合い","スキル見合","ご相談","相談","ご提示下さい","ご提示ください","スキルにより相談"]
    for word in ser_word:
      words = words.replace(word,"")
      
    #曖昧単価加工
    if len(re.findall('[0-9]{1}・',words)) > 0:
      aimai = re.findall('[0-9]{1}・',words)[0]
      words = words.replace(aimai,"")
    if len(re.findall('[0-9]{1}･',words)) > 0:
      aimai = re.findall('[0-9]{1}･',words)[0]
      words = words.replace(aimai,"")
    if len(re.findall('[0-9]{1}、',words)) > 0:
      aimai = re.findall('[0-9]{1}、',words)[0]
      words = words.replace(aimai,"")
      
    #改行コード変換
    words = words.replace("\r\n","")

    #小数点無効
    words = words.replace(".0","")

    #万円単位変換
    words = words.replace("0000","")
    
    #範囲指定正規化(「まで」表現置換)
    #search(最初の一致文字列の数値のみを返す)
    if re.search('[0-9]{2}まで',words):
      made_val = re.sub(r'\D', '',re.search('[0-9]{2}まで',words).group())
      words = words.replace(made_val+'まで','~'+made_val)
      
    return words

In [0]:
def tanka_pickup(str_wk):
  #======================================================
  #スキップワード削除
  #======================================================
  del_word = ["<",">","＜","＞","[","]","《","》","≪","≫","【", "】","(",")","（","）"," ","　",":","：","完全","位","くらい","ぐらい","およそ","大体","だいたい","約","前後","円","万","下さい","ください"]
  
  base_word = '単価'
  tanka_yuragi = ["予算単価","単金","金額","価格","月額","予算"]

  #削除
  for word in del_word:
    str_wk = str_wk.replace(word,'')

  #ゆらぎ統一
  for tanka in tanka_yuragi:
    str_wk = str_wk.replace(tanka,base_word)

  #正規化
  str_wk = serialize(str_wk)

  #======================================================
  #分かち書き
  #======================================================
  tagger = MeCab.Tagger("-Owakati")
  list_output = (tagger.parse(str_wk)).split(' ')

  #======================================================
  #「単価」に最も近い位置で書かれている数字が含まれる文節を抽出
  #======================================================
  tanka_str = ""

  #上限範囲指定
  if list_output.count(base_word) == 1:
    if list_output[list_output.index(base_word)+1] == "~":
      tanka_str = "00~" + list_output[list_output.index(base_word)+2]

  #上限下限指定
  #if list_output.count(base_word) == 1:
  #  if list_output[list_output.index(base_word)+2] == "~":
  #    tanka_str = tanka_str + "~" + list_output[list_output.index(base_word)+3]

  #上限のみ指定
  #★エラー有り
  #if list_output[list_output.index(base_word)+1] == "MAX":
  #  tanka_str ="00~" + list_output[list_output.index(base_word)+2]
    
  if list_output.count(base_word) > 0:
    if list_output[list_output.index(base_word)+1].isdecimal() is True or list_output[list_output.index(base_word)+2].isdecimal() is True or list_output[list_output.index(base_word)+3].isdecimal() is True:
      tanka_str = list_output[list_output.index(base_word)+1] + list_output[list_output.index(base_word)+2] + list_output[list_output.index(base_word)+3]
    else:
      tanka_str = "単価情報あり"
  else:
    tanka_str = "単価完全なし"
    
  return tanka_str

In [90]:
import pandas as pd
import codecs

#utf-8で作られていないファイルの場合のCSV読み込み方法
with codecs.open("python.CSV", "r", "Shift-JIS", "ignore") as file:
    df = pd.read_table(file, delimiter=",")

df2 = df[["本文"]]

#行単位の繰り返し
for index,item in df2.iterrows():
  str_wk = item['本文']
  pickup_str = tanka_pickup(str_wk)

  #======================================================
  #範囲指定単価(exp:A～B) 編集
  #======================================================
  tanka_range = re.findall('[0-9]{2}~[0-9]{2}',pickup_str)

  #下限算出
  if len(tanka_range) == 1:
    min_tanka = int(tanka_range[0].split('~')[0])
  else:
    min_tanka = 0

  #上限算出
  if len(tanka_range) == 1:
    max_tanka = int(tanka_range[0].split('~')[1])
  else:
    max_tanka = 0

  #======================================================
  #上限指定単価(exp:～A) 編集
  #======================================================
  if min_tanka == 0 and max_tanka == 0:
    tanka_range = re.findall('~[0-9]{2}',pickup_str)

    #下限算出
    min_tanka = 0

    #上限算出
    if len(tanka_range) == 1:
      max_tanka = pickup_str.replace("~","") 
      #数値のみ抽出
      max_tanka = re.sub(r'\D', '',max_tanka)

  #======================================================
  #単価単独指定(exp:A) 編集
  #======================================================
  if min_tanka == 0 and max_tanka == 0:
    tanka_range = re.findall('[0-9]{2}',pickup_str)

    #下限算出
    if len(tanka_range) == 1:
      #数値のみ抽出
      min_tanka = re.sub(r'\D', '',tanka_range[0])

    #上限算出
    if len(tanka_range) == 1:
      #数値のみ抽出
      max_tanka = re.sub(r'\D', '',tanka_range[0])
      
  print(str(index) + ":" + pickup_str +"|MIN:" + str(min_tanka) + "|" + "MAX:" + str(max_tanka))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: read_table is deprecated, use read_csv instead.
  


0:~55/|MIN:0|MAX:55
1:単価完全なし|MIN:0|MAX:0
2:単価完全なし|MIN:0|MAX:0
3:単価完全なし|MIN:0|MAX:0
4:単価情報あり|MIN:0|MAX:0
5:65◆枠|MIN:65|MAX:65
6:~85◆|MIN:0|MAX:85
7:~80精算|MIN:0|MAX:80
8:80/月|MIN:80|MAX:80
9:~90◆|MIN:0|MAX:90
10:60~80|MIN:60|MAX:80
11:単価完全なし|MIN:0|MAX:0
12:~75◆|MIN:0|MAX:75
13:85◆面談|MIN:85|MAX:85
14:~95◆|MIN:0|MAX:95
15:~95◆|MIN:0|MAX:95
16:単価情報あり|MIN:0|MAX:0
17:~70◆|MIN:0|MAX:70
18:80/月|MIN:80|MAX:80
19:60~80|MIN:60|MAX:80
20:単価情報あり|MIN:0|MAX:0
21:~85◆|MIN:0|MAX:85
22:65◆枠|MIN:65|MAX:65
23:~90◆|MIN:0|MAX:90
24:~90◆|MIN:0|MAX:90
25:80~90|MIN:80|MAX:90
26:80/月|MIN:80|MAX:80
27:単価情報あり|MIN:0|MAX:0
28:85◆面談|MIN:85|MAX:85
29:~95◆|MIN:0|MAX:95
30:~95◆|MIN:0|MAX:95
31:~75◆|MIN:0|MAX:75
32:~70◆|MIN:0|MAX:70
33:80~90|MIN:80|MAX:90
34:85◆面談|MIN:85|MAX:85
35:単価情報あり|MIN:0|MAX:0
36:~95◆|MIN:0|MAX:95
37:~95◆|MIN:0|MAX:95
38:単価情報あり|MIN:0|MAX:0
39:~75◆|MIN:0|MAX:75
40:~90※|MIN:0|MAX:90
41:~70◆|MIN:0|MAX:70
42:~85※|MIN:0|MAX:85
43:54業種職務|MIN:54|MAX:54
44:単価完全なし|MIN:0|MAX:0
45:単価情報あり|MIN:0|MAX:0
46:~85◆|M

00~55
MIN:0
MAX:55
